# NSGA-II (Non-dominated Sorting Genetic Algorithm II)

NSGA-2 is apopular multi-objective optimization algorithm. It is widely used for solving problems with multiple conflicting objectives. The algorithm incorporates mechanisms for maintaining diversity in the population while converging toward the Pareto-optimal front.

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyprojroot import here
from pathlib import Path

from nsga_2.algo import (
    create_population, objective, assign_fronts, calculate_crowding_distance,
    generate_offspring, flatten_fronts
)

## Parameters

In [10]:
MAX_VALUE = 5
BOUNDS = (-MAX_VALUE, MAX_VALUE)
N_POP = 10
N_GENS = 10

PLOTTING = True

In [11]:
if PLOTTING:
    frames_dir = here("figures/frames")

    if frames_dir.exists():
        for file in frames_dir.iterdir():
            file.unlink()
    else:
        frames_dir.mkdir(parents=True)

## Objectives

In [12]:
f1 = lambda x: x ** 2
f2 = lambda x: (x - 2) ** 2

## Core Loop

In [13]:
p = create_population(N_POP, BOUNDS)

In [15]:
for gen in range(N_GENS):
    print(f"Generation: {gen}")
    
    p_obj = objective(p, (f1, f2))

    # plotting
    if PLOTTING:
        fig, ax = plt.subplots(figsize=(8, 5))
        plt.scatter(x=p_obj[:, 0], y=p_obj[:, 1])
        ax.set(title=f"Generation {gen}", xlabel="$f_1(x)$", ylabel="$f_2(x)$")
        plt.savefig(frames_dir / f"gen_{gen}.png")
        plt.close()

    fronts = assign_fronts(p_obj)

    crowding_distances = calculate_crowding_distance(p_obj)

    Q = generate_offspring(p_obj, p, fronts, crowding_distances, BOUNDS)

    R = np.vstack((p, Q))
    assert R.shape[0] == 2 * N_POP

    r_obj = objective(R, (f1, f2))
    r_fronts = assign_fronts(r_obj)
    r_crowding_distances = calculate_crowding_distance(r_obj)

    r_combined = np.column_stack((R.reshape(-1), flatten_fronts(r_obj, r_fronts), r_crowding_distances))
    sorted_indices = np.lexsort((r_combined[:, -2], -r_combined[:, -1]))
    r_combined = r_combined[sorted_indices]
    r_combined = r_combined[:N_POP, :]
    p = r_combined[:, :p.shape[1]]
    

Generation: 0
Generation: 1
Generation: 2
Generation: 3
Generation: 4
Generation: 5
Generation: 6
Generation: 7
Generation: 8
Generation: 9
